In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def two_good_growth(p_taco_0=2.0, p_tablet_0=500, 
                    p_taco_1=3.0, p_tablet_1=550,
                    q_taco_0=100, q_tablet_0=5, 
                    q_taco_1=120, q_tablet_1=6):

    # GDPs
    nom_0 = p_taco_0 * q_taco_0 + p_tablet_0 * q_tablet_0
    nom_1 = p_taco_1 * q_taco_1 + p_tablet_1 * q_tablet_1
    real_1 = p_taco_0 * q_taco_1 + p_tablet_0 * q_tablet_1

    g_real = (real_1 - nom_0) / nom_0

    print("📊 GDP Calculations")
    print(f"Year 0 Nominal GDP: ${nom_0:,.2f}")
    print(f"Year 1 Nominal GDP: ${nom_1:,.2f}")
    print(f"Year 1 Real GDP (base year prices): ${real_1:,.2f}")
    print(f"➡️ Real GDP Growth: {g_real * 100:.2f}%")

    # Bar chart
    labels = ['Year 0 Nominal', 'Year 1 Nominal', 'Year 1 Real']
    values = [nom_0, nom_1, real_1]
    colors = ['#5DADE2', '#F5B041', '#58D68D']

    plt.figure(figsize=(7,4))
    plt.bar(labels, values, color=colors)
    for i, v in enumerate(values):
        plt.text(i, v + 50, f"${v:,.0f}", ha='center')
    plt.title("Two Goods GDP Comparison")
    plt.ylabel("Total GDP ($)")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

interact(two_good_growth,
         p_taco_0=FloatSlider(value=2.0, min=1.0, max=5.0, step=0.1, description='P₀ Tacos'),
         p_tablet_0=FloatSlider(value=500, min=100, max=1000, step=50, description='P₀ Tablets'),
         p_taco_1=FloatSlider(value=3.0, min=1.0, max=6.0, step=0.1, description='P₁ Tacos'),
         p_tablet_1=FloatSlider(value=550, min=100, max=1100, step=50, description='P₁ Tablets'),
         q_taco_0=FloatSlider(value=100, min=50, max=200, step=10, description='Q₀ Tacos'),
         q_tablet_0=FloatSlider(value=5, min=1, max=10, step=1, description='Q₀ Tablets'),
         q_taco_1=FloatSlider(value=120, min=50, max=200, step=10, description='Q₁ Tacos'),
         q_tablet_1=FloatSlider(value=6, min=1, max=10, step=1, description='Q₁ Tablets'));

interactive(children=(FloatSlider(value=2.0, description='P₀ Tacos', max=5.0, min=1.0), FloatSlider(value=500.…

# 🛒 Two Goods, Two Time Periods

When measuring real GDP growth, we need to separate:
- **Changes in quantities** from
- **Changes in prices**

Let’s use a simple economy with two goods:  
- 🌮 Tacos  
- 📱 Tablets

# 🔢 Real vs. Nominal GDP

We compute:
- **Nominal GDP**: current prices × current quantities
- **Real GDP**: current quantities × base-year prices

\[
\text{Nominal GDP}_t = P_{t}^{\text{taco}} \cdot Q_{t}^{\text{taco}} + P_{t}^{\text{tablet}} \cdot Q_{t}^{\text{tablet}}
\]

\[
\text{Real GDP}_t = P_{0}^{\text{taco}} \cdot Q_{t}^{\text{taco}} + P_{0}^{\text{tablet}} \cdot Q_{t}^{\text{tablet}}
\]

> This shows **growth in real output** while holding prices constant.

# 💡 Why It Matters

If prices double and output stays the same, **nominal GDP doubles** — but **real GDP does not**.  
We care about real GDP to understand **true growth** in standard of living.